In [ ]:
import cv2
import numpy as np
from PIL import Image
import os
import tkinter as tk
from tkinter import filedialog
from transformers import pipeline

class ImageCropper:
    def __init__(self, root):
        self.root = root
        self.root.title("Image Cropper")
        # Initialize the crack detection pipeline
        self.crack_detection_pipeline = pipeline("image-classification", model="Taki3d/CrackDetection")

    def select_image_and_coordinates(self):
        # Ask the user to select an image file
        image_path = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg;*.png;*.gif;*.bmp")])
        
        if not image_path:
            return

        # Ask the user to select a folder containing coordinate files
        folder_path = filedialog.askdirectory(title="Select Folder containing Coordinate Files")

        if not folder_path:
            return

        # Get a list of all text files in the folder
        coordinate_files = [file for file in os.listdir(folder_path) if file.endswith(".txt")]

        # Initialize a variable to store all outlines
        all_outlines = []

        # Iterate through each coordinate file
        for file_name in coordinate_files:
            file_path = os.path.join(folder_path, file_name)
            
            # Process the image with the current coordinate file
            outlines = self.process_image(image_path, file_path)
            all_outlines.extend(outlines)

        # Draw all outlines on the original image
        original_image_with_outlines = self.draw_outlines(image_path, all_outlines, coordinate_files)

        # Save the image with outlines
        outlines_output_folder = os.path.join(os.path.dirname(image_path), "outlines")
        os.makedirs(outlines_output_folder, exist_ok=True)
        outlines_output_filename = f"{os.path.splitext(os.path.basename(image_path))[0]}_with_outlines.jpg"
        outlines_output_path = os.path.join(outlines_output_folder, outlines_output_filename)
        cv2.imwrite(outlines_output_path, cv2.cvtColor(original_image_with_outlines, cv2.COLOR_RGB2BGR))
        print(f"Image with outlines saved to: {outlines_output_path}")

    def process_image(self, image_path, coordinates_file):
        # Load the image using OpenCV
        original_image = cv2.imread(image_path)
        original_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
        image = Image.fromarray(original_image)

        # Read coordinates from file
        with open(coordinates_file, 'r') as file:
            polygon_points = [tuple(map(float, line.strip().split(','))) for line in file]

        # Convert the polygon points to numpy array
        roi_corners = np.array([polygon_points], dtype=np.int32)

        # Create a mask using the polygon
        mask = np.zeros_like(np.array(image), dtype=np.uint8)
        cv2.fillPoly(mask, roi_corners, (255, 255, 255))

        # Apply the mask to the original image
        masked_image = cv2.bitwise_and(original_image, mask)

        # Find contours in the mask
        contours, _ = cv2.findContours(cv2.cvtColor(mask, cv2.COLOR_RGB2GRAY), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        # Initialize a list to store outlines
        outlines = []

        # Iterate through each contour
        for contour in contours:
            # Find the bounding rectangle of the contour
            x, y, w, h = cv2.boundingRect(contour)

            # Crop the region of interest from the original image
            cropped_image = masked_image[y:y+h, x:x+w]

            # Convert the cropped image to a PIL Image
            cropped_image_pil = Image.fromarray(cropped_image)

            # Save the cropped image with the name of the coordinate file
            output_folder = os.path.join(os.path.dirname(image_path), "cropped_images")
            os.makedirs(output_folder, exist_ok=True)
            output_filename = f"{os.path.splitext(os.path.basename(coordinates_file))[0]}_cropped.jpg"
            output_path = os.path.join(output_folder, output_filename)
            cropped_image_pil.save(output_path)
            print(f"Cropped image saved to: {output_path}")

            # Apply crack detection on the cropped image
            crack_detection_result = self.crack_detection_pipeline(output_path)
            print("Crack detection result:", crack_detection_result)

            # Check if crack score > 0.30
            for result in crack_detection_result:
                if result['label'] == 'crack' and result['score'] > 0.30:
                    outlines.append((contour, (255, 0, 0), os.path.splitext(os.path.basename(coordinates_file))[0]))  # Red outline for high crack score
                    break  # Once we find the score for 'crack', we break the loop
        
        return outlines

    def draw_outlines(self, image_path, outlines, coordinate_files):
        # Load the image using OpenCV
        original_image = cv2.imread(image_path)
        original_image_with_outlines = original_image.copy()  # Create a copy for drawing outlines

        # Draw outlines on the original image
        for contour, color, coord_file_name in outlines:
            cv2.drawContours(original_image_with_outlines, [contour], -1, color, 2)

            # Add coordinate file name as text annotation
            cv2.putText(original_image_with_outlines, coord_file_name,
                        (int(contour[:, :, 0].mean()), int(contour[:, :, 1].mean())),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.3, (255, 255, 255), 1, cv2.LINE_AA)

        return original_image_with_outlines

 
def compare_images(input_image_path, template_images, threshold=0.8):
    input_image = cv2.imread(input_image_path)
    input_gray = cv2.cvtColor(input_image, cv2.COLOR_BGR2GRAY)
    input_width, input_height = input_gray.shape[::-1]
    i = 0
    j = 0
    for template_image_path in template_images:
        template_image = cv2.imread(template_image_path, 0)
        w, h = template_image.shape[::-1]
 
        # Apply template matching
        result = cv2.matchTemplate(input_gray, template_image, cv2.TM_CCOEFF_NORMED)
        _, max_val, _, max_loc = cv2.minMaxLoc(result)
        print(max_val)
        i += 1
        if max_val >= threshold:
            j+=1
            print("Match found for template:", template_image_path)
            print("Image matches at index:", i)
    print("Total matches found:", j)
   
 
    # Check if all templates have been processed
    if i == len(template_images):
        print("All templates processed.")
           
 
# Example usage
input_image_path = "C:\\Users\\parth.chaturvedi\\Downloads\\image (2).png"
template_images = ["C:\\Users\\parth.chaturvedi\\Downloads\\image (4).png","C:\\Users\\parth.chaturvedi\\Downloads\\image (2).png"]     #"template2.jpg", "template3.jpg", "template4.jpg"
compare_images(input_image_path, template_images)
 

if __name__ == "__main__":
    # Example usage
    input_image_path = "C:\\Users\\parth.chaturvedi\\Downloads\\image (2).png"
    template_images = ["C:\\Users\\parth.chaturvedi\\Downloads\\image (4).png","C:\\Users\\parth.chaturvedi\\Downloads\\image (2).png"]     #"template2.jpg", "template3.jpg", "template4.jpg"
    compare_images(input_image_path, template_images)
    # root = tk.Tk()
    # cropper = ImageCropper(root)
    # cropper.select_image_and_coordinates()
    # root.mainloop()
